In [20]:
import pandas as pd
import sklearn
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
%matplotlib inline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from utils import update_submission

# RandomForest

1. Bagging (sampling with replacement)
   100 samples, randomly choose 60 out of 100, load them to Tree 1.
   Put the 60 back, resample 60, load the new 60 samples to Tree 2.
   ...
   Tree N.
2. Max_feature:
   Each split on the tree is based on one feature, each tree will have different features, maxed at Max_feature
When making predictions, all the trees will vote.

In [2]:
df_test = pd.read_csv('./data/test.csv')
df_train = pd.read_csv('./data/train.csv')
df_submission = pd.read_csv('./data/gender_submission.csv')

In [3]:
def transform(df,option):
    assert option in ["test","train"] , "Option must be test or train"
    df = df.set_index('PassengerId')
    cond = df.Fare.isnull()
    sub_value = df.Fare.mean()
    df.Fare = np.where(cond, sub_value, df.Fare)
    ticket_p = df.groupby('Ticket', as_index=False).agg({'Name': 'count'}).rename(columns={'Name':'t_count'}).sort_values('t_count', ascending=False)
    d = dict()
    for i, row in ticket_p.iterrows():
        d[row.Ticket] = row.t_count
    df_merge = df.copy()
    df_merge['t_count'] = df_merge.Ticket.apply(lambda x: d[x])
    df_merge['Fare_per_person'] = df_merge.Fare/df_merge.t_count
    cond = df_merge.Age.isnull()
    sub_value = df_merge.Age.mean()
    df_merge.Age = np.where(cond, sub_value, df_merge.Age)
    cond = (df_merge.Sex == 'female')
    sub_value = 1
    df_merge.Sex = np.where(cond, sub_value, 0)
    cond = df_merge.Embarked.isnull()
    sub_value = 'S'
    df_merge.Embarked = np.where(cond, 'S', df_merge.Embarked)
    df_merge = df_merge.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    df_merge.isnull().sum()
    ohe = OneHotEncoder()
    X_cat = ohe.fit_transform(df_merge.Embarked.values.reshape(-1,1)).toarray()
    if option=="train":
        X_short = df_merge.drop(['Survived','Embarked'], axis=1).values
    else:
        X_short = df_merge.drop(['Embarked'], axis=1).values
    X = np.hstack([X_cat, X_short])
    return X

In [4]:
X_train_val = transform(df_train, "train")
X_test = transform(df_test,"test")
y_train_val = df_train.Survived

In [5]:
rf = RandomForestClassifier()

In [6]:
cross_val_score(rf, X_train_val, y_train_val).mean()

0.8002385286548239

In [7]:
# param_grid = {'n_estimators': [50, 100, 150, ...], # 3
#               'max_depth': np.arange(4,10), # 6
#               'min_samples_split': np.arange(10,20,2), # 5
#               'min_samples_leaf': np.arange(1,10,2), # 5
#               'max_features': ?????} # 3

In [8]:
# HW (Jan 12. 2021):
# 1. to fine tune our RandomForestClassifier and get a accuracy > 82%.
# 2. read about (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) 
#    print the confusion matrix of your model on the whole train_val set
# 3. update the submission file with the function in utils.py, submit it to kaggle, compare the result with
# what we get locally (CV)
# 4. Read about https://www.youtube.com/watch?v=bfmFfD2RIcg

In [9]:
# 3*6*5*5*3 --> 50 mins

In [10]:
gb = GradientBoostingClassifier()
cross_val_score(gb, X_train_val, y_train_val).mean()

0.8159688657334756

Type I error (false positive):
model says positive, actually negative (COVID)
Type II error (false negative):
model says negative, actually positive (COVID)

Type II error more harmful than Type I error

100 person, 50/50 (COVID/HC)

model 1 (simply classifies every person to have COVID):
accuracy: 50%
Type II: 0

model 2:
50 COVID: 30 correct, 20 wrong
50 HC: 50 correct, 0 wrong
accuracy: 80%
Type II: 20

model 3:
50 COVID: 45 correct, 5 wrong
50 HC: 25 correct, 25 wrong
accuracy: 70%
Type II: 5

confusion matrix:
           model 1      model 0
true 1        50           0
true 0        50           0

real life COST for mistakes:
(domain knowledge) 

COVID
Type II 100 bucks
Type I 1 buck
model 1 cost: 50 bucks
model 2 cost: 2000 bucks
model 3 cost: 525 bucks

FLU
Type II 5 bucks
Type I 1 buck
model 1 cost: 50 bucks
model 2 cost: 100 bucks
model 3 cost: 50 bucks

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train_val, y_train_val)
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [47]:
y_pred_proba = gb.predict_proba(X_test)
roc_auc_score(y_test, y_pred_proba[:,1])

0.8987373737373737

In [52]:
def downsample(gb, y_test, X_test):
    shadow = np.random.random(len(y_test))
    idx = (y_test == 1) & (shadow > 0.5) | (y_test == 0)
    y_test_s = y_test[idx]
    X_test_s = X_test[idx]
    print(y_test.sum()/len(y_test), y_test_s.sum()/len(y_test_s))
    y_pred_proba_s = gb.predict_proba(X_test_s)
    return roc_auc_score(y_test_s, y_pred_proba_s[:,1])

In [63]:
downsample(gb, y_test, X_test)

0.39461883408071746 0.25


0.8882304526748971